In [34]:
# Imports necessários
import pandas as pd
import numpy as np
import psycopg2
from psycopg2.extras import execute_values
from datetime import datetime, date
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

print("✅ Bibliotecas importadas com sucesso!")
print(f"📅 Execução do ETL: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

✅ Bibliotecas importadas com sucesso!
📅 Execução do ETL: 2025-11-23 20:20:14


---
## 1️⃣ Conexão com PostgreSQL

In [35]:
# Configurações de conexão
DB_CONFIG = {
    'dbname': 'movies_dw',
    'user': 'postgres',
    'password': 'postgres123',
    'host': 'localhost',
    'port': '5432'
}

print("🔌 Conectando ao PostgreSQL...")
try:
    conn = psycopg2.connect(**DB_CONFIG)
    cursor = conn.cursor()
    print("   ✅ Conexão estabelecida com sucesso!\n")
    
    # Verificar registros em SILVER
    cursor.execute("SELECT COUNT(*) FROM silver.movies_raw;")
    count_silver = cursor.fetchone()[0]
    print(f"📊 Registros na camada SILVER: {count_silver:,}\n")
    
except Exception as e:
    print(f"❌ Erro na conexão: {e}")
    raise

🔌 Conectando ao PostgreSQL...
   ✅ Conexão estabelecida com sucesso!

📊 Registros na camada SILVER: 45,433



---
## 2️⃣ Carregar Dados da Camada SILVER

In [36]:
print("📥 Carregando dados de silver.movies_raw...")

query = """
SELECT 
    id, title, original_title, original_language,
    release_date, release_year, release_month, release_decade,
    budget, revenue, profit, roi,
    budget_category, revenue_category,
    runtime, runtime_category,
    vote_average, vote_count, popularity,
    genres_list, primary_genre,
    production_companies_list, primary_company,
    production_countries_list, primary_country,
    status, adult,
    overview, tagline, homepage,
    imdb_id, poster_path,
    director, lead_actor, top_actors, cast_size, crew_size,
    keywords_list, keywords_count,
    avg_rating, median_rating, std_rating, total_ratings,
    min_rating, max_rating, unique_users,
    tmdb_id, imdb_id_formatted
FROM silver.movies_raw
WHERE id IS NOT NULL
ORDER BY id;
"""

df_silver = pd.read_sql(query, conn)
print(f"✅ {len(df_silver):,} filmes carregados da camada SILVER\n")
print(f"📊 Colunas disponíveis: {df_silver.shape[1]}")
df_silver.head(3)

📥 Carregando dados de silver.movies_raw...
✅ 45,433 filmes carregados da camada SILVER

📊 Colunas disponíveis: 48
✅ 45,433 filmes carregados da camada SILVER

📊 Colunas disponíveis: 48


,id,title,original_title,original_language,release_date,release_year,release_month,release_decade,budget,revenue,profit,roi,budget_category,revenue_category,runtime,runtime_category,vote_average,vote_count,popularity,genres_list,primary_genre,production_companies_list,primary_company,production_countries_list,primary_country,status,adult,overview,tagline,homepage,imdb_id,poster_path,director,lead_actor,top_actors,cast_size,crew_size,keywords_list,keywords_count,avg_rating,median_rating,std_rating,total_ratings,min_rating,max_rating,unique_users,tmdb_id,imdb_id_formatted
0,2,Ariel,Ariel,fi,1988-10-21,1988.0,10.0,1980.0,0,0.0,0.0,NaN,Micro,Muito Baixa,69.0,Curto,7.1,44.0,3.860,Drama|Crime,Drama,Villealfa Filmproduction Oy|Finnish Film Found...,Villealfa Filmproduction Oy,Finland,Finland,Released,False,Taisto Kasurinen is a Finnish coal miner whose...,None,None,tt0094675,/gZCJZOn4l0Zj5hAxsMbxoS6CL0u.jpg,Aki Kaurismäki,Turo Pajala,Turo Pajala|Susanna Haavisto|Matti Pellonpää|E...,4.0,6.0,underdog|prison|factory worker|prisoner|helsin...,7.0,3.67,4.0,0.91,262.0,0.5,5.0,262.0,2,tt0094675
1,3,Shadows in Paradise,Varjoja paratiisissa,fi,1986-10-16,1986.0,10.0,1980.0,0,0.0,0.0,NaN,Micro,Muito Baixa,76.0,Curto,7.1,35.0,2.292,Drama|Comedy,Drama,Villealfa Filmproduction Oy,Villealfa Filmproduction Oy,Finland,Finland,Released,False,"An episode in the life of Nikander, a garbage ...",None,None,tt0092149,/7ad4iku8cYBuB08g9yAU7tHJik5.jpg,Aki Kaurismäki,Matti Pellonpää,Matti Pellonpää|Kati Outinen|Sakari Kuosmanen|...,7.0,11.0,salesclerk|helsinki|garbage|independent film,4.0,3.77,4.0,0.81,87.0,0.5,5.0,87.0,3,tt0092149
2,5,Four Rooms,Four Rooms,en,1995-12-09,1995.0,12.0,1990.0,4000000,4300000.0,300000.0,7.5,Baixo,Baixa,98.0,Médio,6.5,539.0,9.027,Crime|Comedy,Crime,Miramax Films|A Band Apart,Miramax Films,United States of America,United States of America,Released,False,It's Ted the Bellhop's first night on the job....,Twelve outrageous guests. Four scandalous requ...,None,tt0113101,/eQs5hh9rxrk1m4xHsIz1w11Ngqb.jpg,Allison Anders,Tim Roth,Tim Roth|Antonio Banderas|Jennifer Beals|Madon...,24.0,88.0,hotel|new year's eve|witch|bet|hotel room|sper...,10.0,3.41,3.5,1.03,6090.0,0.5,5.0,6090.0,5,tt0113101


---
## 3️⃣ Limpar Tabelas GOLD (TRUNCATE)

In [37]:
print("🗑️  Limpando tabelas GOLD...\n")

tables = [
    'gold.fto_filme',
    'gold.dim_filme',
    'gold.dim_tempo',
    'gold.dim_genero',
    'gold.dim_companhia',
    'gold.dim_geografia',
    'gold.dim_diretor',
    'gold.dim_ator'
]

for table in tables:
    try:
        cursor.execute(f"TRUNCATE TABLE {table} RESTART IDENTITY CASCADE;")
        conn.commit()
        print(f"   ✅ {table} limpa")
    except Exception as e:
        print(f"   ⚠️  {table}: {e}")
        conn.rollback()

print("\n✅ Limpeza concluída!\n")

🗑️  Limpando tabelas GOLD...

   ✅ gold.fto_filme limpa
   ✅ gold.dim_filme limpa
   ✅ gold.dim_tempo limpa
   ✅ gold.dim_filme limpa
   ✅ gold.dim_tempo limpa
   ✅ gold.dim_genero limpa
   ✅ gold.dim_companhia limpa
   ✅ gold.dim_genero limpa
   ✅ gold.dim_companhia limpa
   ✅ gold.dim_geografia limpa
   ✅ gold.dim_diretor limpa
   ✅ gold.dim_geografia limpa
   ✅ gold.dim_diretor limpa
   ✅ gold.dim_ator limpa

✅ Limpeza concluída!

   ✅ gold.dim_ator limpa

✅ Limpeza concluída!



---
## 4️⃣ Popular DIM_TEMPO

In [38]:
print("📅 Populando dim_tempo...\n")

# Obter datas únicas
datas_unicas = df_silver['release_date'].dropna().unique()
print(f"   {len(datas_unicas):,} datas únicas encontradas")

# Criar DataFrame de tempo
dim_tempo_data = []

meses_pt = {
    1: ('Janeiro', 'JAN'), 2: ('Fevereiro', 'FEV'), 3: ('Março', 'MAR'),
    4: ('Abril', 'ABR'), 5: ('Maio', 'MAI'), 6: ('Junho', 'JUN'),
    7: ('Julho', 'JUL'), 8: ('Agosto', 'AGO'), 9: ('Setembro', 'SET'),
    10: ('Outubro', 'OUT'), 11: ('Novembro', 'NOV'), 12: ('Dezembro', 'DEZ')
}

dias_semana = {
    0: 'Segunda-feira', 1: 'Terça-feira', 2: 'Quarta-feira',
    3: 'Quinta-feira', 4: 'Sexta-feira', 5: 'Sábado', 6: 'Domingo'
}

for data_str in datas_unicas:
    data = pd.to_datetime(data_str)
    
    tmp_srk = int(data.strftime('%Y%m%d'))
    ano = data.year
    mes = data.month
    dia = data.day
    dia_semana = data.weekday()
    trimestre = (mes - 1) // 3 + 1
    semana = data.isocalendar()[1]
    decada = (ano // 10) * 10
    
    mes_nome, mes_abrev = meses_pt[mes]
    
    dim_tempo_data.append((
        tmp_srk, data.date(), ano, mes, mes_nome, mes_abrev,
        dia, dia_semana + 1, dias_semana[dia_semana],
        trimestre, f'T{trimestre}', semana, decada,
        'N', 'S' if dia_semana >= 5 else 'N'
    ))

# Inserir dados
insert_sql = """
INSERT INTO gold.dim_tempo (
    tmp_srk, dta_completa, ano_lancamento, mes_numero, mes_nome, mes_abrev,
    dia_numero, dia_semana, nom_dia_semana, tri_numero, tri_nome, sem_numero,
    dec_inicio, flg_feriado, flg_fim_semana
) VALUES %s
ON CONFLICT (tmp_srk) DO NOTHING;
"""

execute_values(cursor, insert_sql, dim_tempo_data, page_size=1000)
conn.commit()

cursor.execute("SELECT COUNT(*) FROM gold.dim_tempo;")
count = cursor.fetchone()[0]
print(f"\n✅ {count:,} registros inseridos em dim_tempo\n")

📅 Populando dim_tempo...

   17,333 datas únicas encontradas

✅ 17,333 registros inseridos em dim_tempo


✅ 17,333 registros inseridos em dim_tempo



---
## 5️⃣ Popular DIM_GENERO

In [39]:
print("🎬 Populando dim_genero...\n")

# Extrair gêneros únicos
generos = set()
for genres_list in df_silver['primary_genre'].dropna():
    if genres_list:
        generos.add(genres_list)

# Adicionar "Desconhecido"
generos.add('Desconhecido')
print(f"   {len(generos)} gêneros únicos encontrados")

# Preparar dados
dim_genero_data = [(g, None, None) for g in sorted(generos)]

# Inserir
insert_sql = """
INSERT INTO gold.dim_genero (gnr_nome, gnr_descricao, gnr_categoria)
VALUES %s
ON CONFLICT (gnr_nome) DO NOTHING;
"""

execute_values(cursor, insert_sql, dim_genero_data)
conn.commit()

cursor.execute("SELECT COUNT(*) FROM gold.dim_genero;")
count = cursor.fetchone()[0]
print(f"\n✅ {count:,} gêneros inseridos em dim_genero\n")

🎬 Populando dim_genero...

   21 gêneros únicos encontrados

✅ 21 gêneros inseridos em dim_genero



---
## 6️⃣ Popular DIM_COMPANHIA

In [40]:
print("🏢 Populando dim_companhia...\n")

# Extrair companhias únicas
companhias = set()
for company in df_silver['primary_company'].dropna():
    if company:
        companhias.add(company)

companhias.add('Desconhecida')
print(f"   {len(companhias):,} companhias únicas encontradas")

# Preparar dados
dim_companhia_data = [(c, 'Produtora') for c in sorted(companhias)]

# Inserir
insert_sql = """
INSERT INTO gold.dim_companhia (cmp_nome, cmp_tipo)
VALUES %s;
"""

execute_values(cursor, insert_sql, dim_companhia_data)
conn.commit()

cursor.execute("SELECT COUNT(*) FROM gold.dim_companhia;")
count = cursor.fetchone()[0]
print(f"\n✅ {count:,} companhias inseridas em dim_companhia\n")

🏢 Populando dim_companhia...

   10,612 companhias únicas encontradas
   10,612 companhias únicas encontradas

✅ 10,612 companhias inseridas em dim_companhia


✅ 10,612 companhias inseridas em dim_companhia



---
## 7️⃣ Popular DIM_GEOGRAFIA

In [41]:
print("🌍 Populando dim_geografia...\n")

# Extrair países únicos
paises = set()
for country in df_silver['primary_country'].dropna():
    if country:
        paises.add(country)

paises.add('Desconhecido')
print(f"   {len(paises):,} países únicos encontrados")

# Preparar dados (nome, iso, continente, regiao)
dim_geografia_data = [(p, None, None, None) for p in sorted(paises)]

# Inserir
insert_sql = """
INSERT INTO gold.dim_geografia (geo_pais, geo_codigo_iso, geo_continente, geo_regiao)
VALUES %s
ON CONFLICT (geo_pais) DO NOTHING;
"""

execute_values(cursor, insert_sql, dim_geografia_data)
conn.commit()

cursor.execute("SELECT COUNT(*) FROM gold.dim_geografia;")
count = cursor.fetchone()[0]
print(f"\n✅ {count:,} países inseridos em dim_geografia\n")

🌍 Populando dim_geografia...

   144 países únicos encontrados

✅ 144 países inseridos em dim_geografia



---
## 8️⃣ Popular DIM_DIRETOR

In [42]:
print("🎬 Populando dim_diretor...\n")

# Extrair diretores únicos
diretores = set()
for diretor in df_silver['director'].dropna():
    if diretor:
        diretores.add(diretor)

diretores.add('Desconhecido')
print(f"   {len(diretores):,} diretores únicos encontrados")

# Preparar dados
dim_diretor_data = [(d, d) for d in sorted(diretores)]

# Inserir
insert_sql = """
INSERT INTO gold.dim_diretor (dir_nome, dir_nome_completo)
VALUES %s;
"""

execute_values(cursor, insert_sql, dim_diretor_data)
conn.commit()

cursor.execute("SELECT COUNT(*) FROM gold.dim_diretor;")
count = cursor.fetchone()[0]
print(f"\n✅ {count:,} diretores inseridos em dim_diretor\n")

🎬 Populando dim_diretor...

   17,572 diretores únicos encontrados

✅ 17,572 diretores inseridos em dim_diretor


✅ 17,572 diretores inseridos em dim_diretor



---
## 9️⃣ Popular DIM_ATOR

In [43]:
print("🎭 Populando dim_ator...\n")

# Extrair atores únicos
atores = set()
for ator in df_silver['lead_actor'].dropna():
    if ator:
        atores.add(ator)

atores.add('Desconhecido')
print(f"   {len(atores):,} atores únicos encontrados")

# Preparar dados
dim_ator_data = [(a, a) for a in sorted(atores)]

# Inserir
insert_sql = """
INSERT INTO gold.dim_ator (act_nome, act_nome_completo)
VALUES %s;
"""

execute_values(cursor, insert_sql, dim_ator_data)
conn.commit()

cursor.execute("SELECT COUNT(*) FROM gold.dim_ator;")
count = cursor.fetchone()[0]
print(f"\n✅ {count:,} atores inseridos em dim_ator\n")

🎭 Populando dim_ator...

   18,374 atores únicos encontrados

✅ 18,374 atores inseridos em dim_ator


✅ 18,374 atores inseridos em dim_ator



---
## 🔟 Popular DIM_FILME

In [44]:
print("🎞️  Populando dim_filme...\n")

# Preparar dados
dim_filme_data = []

for _, row in df_silver.iterrows():
    # Garantir que mov_titulo nunca seja NULL
    titulo = row['title'] if pd.notna(row['title']) else row['original_title']
    
    dim_filme_data.append((
        int(row['id']),  # mov_nky
        titulo,
        row['original_title'],
        row['original_language'],
        row['imdb_id_formatted'],
        row['overview'],
        row['tagline'],
        row['homepage'],
        row['poster_path'],
        row['status'],
        row['budget_category'],
        row['revenue_category'],
        row['runtime_category']
    ))

# Inserir
insert_sql = """
INSERT INTO gold.dim_filme (
    mov_nky, mov_titulo, mov_titulo_original, cod_idioma, cod_imdb,
    txt_sinopse, txt_tagline, url_homepage, url_poster,
    cat_status, cat_orcamento, cat_receita, cat_duracao
) VALUES %s;
"""

execute_values(cursor, insert_sql, dim_filme_data, page_size=1000)
conn.commit()

cursor.execute("SELECT COUNT(*) FROM gold.dim_filme;")
count = cursor.fetchone()[0]
print(f"\n✅ {count:,} filmes inseridos em dim_filme\n")

🎞️  Populando dim_filme...


✅ 45,433 filmes inseridos em dim_filme


✅ 45,433 filmes inseridos em dim_filme



---
## 1️⃣1️⃣ Criar Dicionários de Mapeamento (Surrogate Keys)

In [45]:
print("🔑 Criando dicionários de mapeamento...\n")

# DIM_FILME (mov_nky -> mov_srk)
cursor.execute("SELECT mov_srk, mov_nky FROM gold.dim_filme;")
map_filme = {nky: srk for srk, nky in cursor.fetchall()}
print(f"   ✅ dim_filme: {len(map_filme):,} mapeamentos")

# DIM_TEMPO (data -> tmp_srk)
cursor.execute("SELECT tmp_srk, dta_completa FROM gold.dim_tempo;")
map_tempo = {data: srk for srk, data in cursor.fetchall()}
print(f"   ✅ dim_tempo: {len(map_tempo):,} mapeamentos")

# DIM_GENERO (nome -> srk)
cursor.execute("SELECT gnr_srk, gnr_nome FROM gold.dim_genero;")
map_genero = {nome: srk for srk, nome in cursor.fetchall()}
print(f"   ✅ dim_genero: {len(map_genero):,} mapeamentos")

# DIM_COMPANHIA (nome -> srk)
cursor.execute("SELECT cmp_srk, cmp_nome FROM gold.dim_companhia;")
map_companhia = {nome: srk for srk, nome in cursor.fetchall()}
print(f"   ✅ dim_companhia: {len(map_companhia):,} mapeamentos")

# DIM_GEOGRAFIA (pais -> srk)
cursor.execute("SELECT geo_srk, geo_pais FROM gold.dim_geografia;")
map_geografia = {pais: srk for srk, pais in cursor.fetchall()}
print(f"   ✅ dim_geografia: {len(map_geografia):,} mapeamentos")

# DIM_DIRETOR (nome -> srk)
cursor.execute("SELECT dir_srk, dir_nome FROM gold.dim_diretor;")
map_diretor = {nome: srk for srk, nome in cursor.fetchall()}
print(f"   ✅ dim_diretor: {len(map_diretor):,} mapeamentos")

# DIM_ATOR (nome -> srk)
cursor.execute("SELECT act_srk, act_nome FROM gold.dim_ator;")
map_ator = {nome: srk for srk, nome in cursor.fetchall()}
print(f"   ✅ dim_ator: {len(map_ator):,} mapeamentos\n")

🔑 Criando dicionários de mapeamento...

   ✅ dim_filme: 45,433 mapeamentos
   ✅ dim_tempo: 17,333 mapeamentos
   ✅ dim_genero: 21 mapeamentos
   ✅ dim_companhia: 10,612 mapeamentos
   ✅ dim_geografia: 144 mapeamentos
   ✅ dim_diretor: 17,572 mapeamentos
   ✅ dim_ator: 18,374 mapeamentos

   ✅ dim_diretor: 17,572 mapeamentos
   ✅ dim_ator: 18,374 mapeamentos



---
## 1️⃣2️⃣ Popular FTO_FILME (Tabela Fato)

In [46]:
print("⭐ Populando fto_filme (Tabela Fato)...\n")

fto_filme_data = []
erros = 0

for idx, row in df_silver.iterrows():
    try:
        # Obter foreign keys
        mov_fky = map_filme.get(int(row['id']))
        
        # Tempo
        data_release = pd.to_datetime(row['release_date']).date() if pd.notna(row['release_date']) else None
        tmp_fky = map_tempo.get(data_release, map_tempo.get(date(1900, 1, 1)))
        
        # Dimensões
        gnr_fky = map_genero.get(row['primary_genre'] if pd.notna(row['primary_genre']) else 'Desconhecido', map_genero['Desconhecido'])
        cmp_fky = map_companhia.get(row['primary_company'] if pd.notna(row['primary_company']) else 'Desconhecida', map_companhia['Desconhecida'])
        geo_fky = map_geografia.get(row['primary_country'] if pd.notna(row['primary_country']) else 'Desconhecido', map_geografia['Desconhecido'])
        dir_fky = map_diretor.get(row['director'] if pd.notna(row['director']) else 'Desconhecido', map_diretor['Desconhecido'])
        act_fky = map_ator.get(row['lead_actor'] if pd.notna(row['lead_actor']) else 'Desconhecido', map_ator['Desconhecido'])
        
        if not mov_fky or not tmp_fky:
            erros += 1
            continue
        
        # Métricas
        fto_filme_data.append((
            mov_fky, tmp_fky, gnr_fky, cmp_fky, geo_fky, dir_fky, act_fky,
            row['budget'] if pd.notna(row['budget']) else None,
            row['revenue'] if pd.notna(row['revenue']) else None,
            row['profit'] if pd.notna(row['profit']) else None,
            row['roi'] if pd.notna(row['roi']) else None,
            row['avg_rating'] if pd.notna(row['avg_rating']) else None,
            int(row['vote_count']) if pd.notna(row['vote_count']) else None,
            row['popularity'] if pd.notna(row['popularity']) else None,
            row['runtime'] if pd.notna(row['runtime']) else None,
            int(row['cast_size']) if pd.notna(row['cast_size']) else None,
            int(row['crew_size']) if pd.notna(row['crew_size']) else None,
            'S' if row['adult'] == True else 'N',
            'S' if row['budget_category'] == 'Blockbuster' else 'N'
        ))
        
        if (idx + 1) % 10000 == 0:
            print(f"   📦 Processado: {idx + 1:,} registros...")
            
    except Exception as e:
        erros += 1
        continue

print(f"\n   ✅ {len(fto_filme_data):,} registros preparados")
print(f"   ⚠️  {erros} erros/ignorados\n")

# Inserir
insert_sql = """
INSERT INTO gold.fto_filme (
    mov_fky, tmp_fky, gnr_fky, cmp_fky, geo_fky, dir_fky, act_fky,
    vlr_orcamento, vlr_receita, vlr_lucro, pct_roi,
    med_avaliacao, qtd_votos, med_popularidade,
    dur_minutos, qtd_elenco, qtd_equipe,
    flg_adulto, flg_blockbuster
) VALUES %s;
"""

execute_values(cursor, insert_sql, fto_filme_data, page_size=1000)
conn.commit()

cursor.execute("SELECT COUNT(*) FROM gold.fto_filme;")
count = cursor.fetchone()[0]
print(f"\n✅ {count:,} fatos inseridos em fto_filme\n")

⭐ Populando fto_filme (Tabela Fato)...

   📦 Processado: 10,000 registros...
   📦 Processado: 10,000 registros...
   📦 Processado: 20,000 registros...
   📦 Processado: 20,000 registros...
   📦 Processado: 30,000 registros...
   📦 Processado: 30,000 registros...
   📦 Processado: 40,000 registros...
   📦 Processado: 40,000 registros...

   ✅ 45,433 registros preparados
   ⚠️  0 erros/ignorados


   ✅ 45,433 registros preparados
   ⚠️  0 erros/ignorados


✅ 45,433 fatos inseridos em fto_filme


✅ 45,433 fatos inseridos em fto_filme



---
## 1️⃣3️⃣ Validação Final

In [47]:
print("🔍 Validando dados carregados na camada GOLD...\n")
print("="*80)

tabelas = [
    'gold.dim_tempo',
    'gold.dim_genero',
    'gold.dim_companhia',
    'gold.dim_geografia',
    'gold.dim_diretor',
    'gold.dim_ator',
    'gold.dim_filme',
    'gold.fto_filme'
]

for tabela in tabelas:
    cursor.execute(f"SELECT COUNT(*) FROM {tabela};")
    count = cursor.fetchone()[0]
    print(f"   {tabela:30s}: {count:>10,} registros")

print("="*80)

# Exemplo de consulta analítica
print("\n📊 Top 5 Gêneros por Receita Total:\n")
query = """
SELECT 
    g.gnr_nome,
    COUNT(*) as total_filmes,
    ROUND(SUM(f.vlr_receita)::numeric / 1000000, 2) as receita_total_milhoes
FROM gold.fto_filme f
JOIN gold.dim_genero g ON f.gnr_fky = g.gnr_srk
WHERE f.vlr_receita > 0
GROUP BY g.gnr_nome
ORDER BY receita_total_milhoes DESC
LIMIT 5;
"""

df_top_generos = pd.read_sql(query, conn)
print(df_top_generos.to_string(index=False))

# Fechar conexão
cursor.close()
conn.close()
print("\n🔌 Conexão com PostgreSQL encerrada.")

print("\n" + "="*80)
print("✅ ETL SILVER → GOLD CONCLUÍDO COM SUCESSO!")
print("="*80)

🔍 Validando dados carregados na camada GOLD...

   gold.dim_tempo                :     17,333 registros
   gold.dim_genero               :         21 registros
   gold.dim_companhia            :     10,612 registros
   gold.dim_geografia            :        144 registros
   gold.dim_diretor              :     17,572 registros
   gold.dim_ator                 :     18,374 registros
   gold.dim_filme                :     45,433 registros
   gold.fto_filme                :     45,433 registros

📊 Top 5 Gêneros por Receita Total:

 gnr_nome  total_filmes  receita_total_milhoes
   Action          1195              122047.61
Adventure           474               84130.60
   Comedy          1568               75816.86
    Drama          1934               72536.17
Animation           188               35852.27

🔌 Conexão com PostgreSQL encerrada.

✅ ETL SILVER → GOLD CONCLUÍDO COM SUCESSO!
